In [104]:
import pandas as pd

In [105]:
# -------------------
# 1. 개별 결과 불러오기
# -------------------
cri_big5 = pd.read_csv("../result/CRI/CRI-BIG5CRIME_result.csv")
cri_cctv = pd.read_csv("../result/CRI/CRI-CCTV_result.csv")

drm_civ = pd.read_csv("../result/DRM/DRM-CIVDEF_result.csv")
drm_eme = pd.read_csv("../result/DRM/DRM-EMERGWATER_result.csv")
drm_flo = pd.read_csv("../result/DRM/DRM-FLOODSHELTER_result.csv")
drm_pms = pd.read_csv("../result/DRM/DRM-PMSHELTER_result.csv")
drm_qos = pd.read_csv("../result/DRM/DRM-QUAKEOUTSHELTER_result.csv")
drm_qks = pd.read_csv("../result/DRM/DRM-QUAKESHELTER_result.csv")

fir_fir = pd.read_csv("../result/FIR/FIR-FIRESTATION_result.csv")

med_med = pd.read_csv("../result/MED/MED-MEDINST_result.csv")
med_pha = pd.read_csv("../result/MED/MED-PHARM_result.csv")
med_phc = pd.read_csv("../result/MED/MED-PHCENTER_result.csv")

rst_cli = pd.read_csv("../result/RST/RST-CLIMATESHELTER_result.csv")
rst_col = pd.read_csv("../result/RST/RST-COLDSHELTER_result.csv")
rst_hea = pd.read_csv("../result/RST/RST-HEATSHELTER_result.csv")
rst_lib = pd.read_csv("../result/RST/RST-LIBSHELTER_result.csv")
rst_sma = pd.read_csv("../result/RST/RST-SMARTSHELTER_result.csv")

tra_sub = pd.read_csv("../result/TRA/TRA-SUBCONGEST_result.csv")
tra_acc = pd.read_csv("../result/TRA/TRA-TRAACCIDENT_result.csv")
tra_saf = pd.read_csv("../result/TRA/TRA-TRASAFETYIDX_result.csv")

In [106]:
# -------------------
# 1. 범죄 점수
# -------------------

# 1-1. CRI-BIG5CRIME 점수 만들기
# (발생 건수 점수와 검거율 점수를 단순 평균)
cri_big5["big5crime_score"] = (
    cri_big5["big5crime_occurrence_score"] + cri_big5["big5crime_arrest_score"]
) / 2

# 1-2. CCTV 점수 컬럼 이름 확인 후 정리
# (파일에 따라 'score' 또는 다른 이름일 수 있음)
cri_cctv = cri_cctv.rename(columns={"score": "cctv_score"})

# 1-3. 자치구 기준 merge
cri = cri_big5.merge(cri_cctv, on="자치구")

# 1-4. 최종 범죄 점수 (가중 평균: Big5 0.7, CCTV 0.3)
cri["CRI_score"] = 0.7 * cri["big5crime_score"] + 0.3 * cri["cctv_score"]


In [107]:
# -------------------
# 2. 재난 점수
# -------------------
drm = drm_civ.merge(drm_eme, on="자치구")
drm = drm.merge(drm_flo, on="자치구").merge(drm_pms, on="자치구")
drm = drm.merge(drm_qos, on="자치구").merge(drm_qks, on="자치구")
drm["DRM_score"] = 0.5*((drm["civdef_mean_score"]+drm["emergwater_mean_score"])/2) + \
                   0.5*((drm["floodshelter_mean_score"]+drm["pmshelter_mean_score"]+
                         drm["quakeoutshelter_mean_score"]+drm["quakeshelter_mean_score"])/4)

In [108]:
# -------------------
# 3. 소방 점수
# -------------------
fir = fir_fir.rename(columns={"firestation_mean_score":"FIR_score"})

In [109]:
# -------------------
# 4. 의료 점수
# -------------------
med = med_med.merge(med_pha, on="자치구").merge(med_phc, on="자치구")
med["MED_score"] = 0.5*med["medinst_mean_score"] + \
                   0.25*med["pharmacy_score"] + 0.25*med["phcenter_mean_score"]

In [110]:
rst_sma.shape

(10, 3)

In [111]:
# -------------------
# 5. 쉼터 점수 - 스마트쉼터는 결측값이 많으므로 제외
# -------------------
rst = rst_cli.merge(rst_col, on="자치구").merge(rst_hea, on="자치구")
rst = rst.merge(rst_lib, on="자치구")
rst["RST_score"] = 0.4*rst["climateshelter_mean_score"] + \
                   0.4*rst["coldshelter_mean_score"] + \
                   0.1*rst["heatshelter_mean_score"] + \
                   0.1*rst["libshelter_score"]

In [112]:
tra_acc.columns

Index(['자치구', '발생건수', '사망자수', '부상자수', 'traaccident_score'], dtype='object')

In [113]:
# -------------------
# 6. 교통 점수
# -------------------
tra = tra_sub.merge(tra_acc, on="자치구").merge(tra_saf, on="자치구")
tra["TRA_score"] = 0.6*tra["교통사고위험도_score"] + \
                   0.3*tra["traaccident_score"] + 0.1*tra["subway_congest_score"]

In [114]:
# -------------------
# 7. 최종 통합
# -------------------
dfs = [cri[["자치구", "CRI_score"]],
       drm[["자치구", "DRM_score"]],
       fir[["자치구", "FIR_score"]],
       med[["자치구", "MED_score"]],
       rst[["자치구", "RST_score"]],
       tra[["자치구", "TRA_score"]]]

In [115]:
final = dfs[0]
for df in dfs[1:]:
    final = final.merge(df, on="자치구")

final["safety_score"] = (0.2*final["CRI_score"] +
                       0.25*final["DRM_score"] +
                       0.25*final["TRA_score"] +
                       0.1*final["FIR_score"] +
                       0.1*final["MED_score"] +
                       0.1*final["RST_score"])

In [116]:
# -------------------
# 8. 결과 저장
# -------------------
final.to_csv("../result/safety_total_score.csv", index=False)